In [1]:
import pandas as pd

# Data Cleaning

In [2]:
df_historical_data = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

Cleaning df_fixture

In [3]:
df_fixture

,home,score,away,year
0,Qatar,0–2,Ecuador,2022
1,Senegal,0–2,Netherlands,2022
2,Qatar,1–3,Senegal,2022
3,Netherlands,1–1,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [4]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

# Cleaning df_missing_data and additing it to df_historical_data

In [5]:
#null data
df_missing_data[df_missing_data['home'].isnull()]

,home,score,away,year
348,NaN,NaN,NaN,2010
349,NaN,NaN,NaN,2010
350,NaN,NaN,NaN,2010
351,NaN,NaN,NaN,2010
352,NaN,NaN,NaN,2010
...,...,...,...,...
407,NaN,NaN,NaN,2010
408,NaN,NaN,NaN,2010
409,NaN,NaN,NaN,2010
410,NaN,NaN,NaN,2010


In [6]:
df_missing_data.dropna(inplace=True)

In [7]:
#concatenate and clean
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
515,Brazil,2–0,Costa Rica,2018
516,Serbia,1–2,Switzerland,2018
517,Serbia,0–2,Brazil,2018
519,Germany,0–1,Mexico,2018


# Cleaning df_historical_data

In [8]:
#Deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden')
                                  & df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

In [9]:
#Cleaning score columns with onlt digits
# df_historical_data['score'].str.contains('[^\d–]')
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)


#Cleaning home/away columns
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [10]:
#Splitting score columns into home & away goals and dropping score column
df_historical_data[['HomeGoals','AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1,inplace=True)

In [11]:
df_historical_data.dtypes

home         object
away         object
year          int64
HomeGoals    object
AwayGoals    object
dtype: object

In [13]:
# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

In [16]:
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals'] 

In [17]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
515,Brazil,Costa Rica,2018,2,0,2
516,Serbia,Switzerland,2018,1,2,3
517,Serbia,Brazil,2018,0,2,2
519,Germany,Mexico,2018,0,1,1


# Exporting Clean DataFrames

In [18]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv', index = False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index = False)

# Verifications

In [20]:
#Verify number of matches per competition
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


In [21]:
#Verify data collected for a team
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Germany')])

         HomeTeam    AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
28        Germany      Sweden  1934          2          1           3
33        Germany     Austria  1934          3          2           5
25        Germany     Belgium  1934          5          2           7
96   West Germany  Yugoslavia  1954          2          0           2
98   West Germany     Austria  1954          6          1           7
..            ...         ...   ...        ...        ...         ...
488       Germany   Argentina  2014          1          0           1
461       Germany    Portugal  2014          4          0           4
463       Germany       Ghana  2014          2          2           4
522       Germany      Sweden  2018          2          1           3
519       Germany      Mexico  2018          0          1           1

[75 rows x 6 columns]


In [22]:
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Germany')])

           HomeTeam      AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
32   Czechoslovakia       Germany  1934          3          1           4
43      Switzerland       Germany  1938          4          2           6
35      Switzerland       Germany  1938          1          1           2
82          Hungary  West Germany  1954          8          3          11
134          Sweden  West Germany  1958          3          1           4
135          France  West Germany  1958          6          3           9
102       Argentina  West Germany  1958          1          3           4
164      Yugoslavia  West Germany  1962          1          0           1
200         England  West Germany  1966          4          2           6
178       Argentina  West Germany  1966          0          0           0
206           Italy  West Germany  1970          4          3           7
207         Uruguay  West Germany  1970          0          1           1
602          Brazil  East Germany  197